[View in Colaboratory](https://colab.research.google.com/github/mhmmdshirazi/laser_measurement/blob/master/ZJU.ipynb)

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!wget https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!dpkg -i google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!apt-get install -f
!apt-get -y install -qq fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
import cv2 as cv
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!pip install -q torch
!pip install -q torchvision
!pip install -q scipy
!pip install -q numpy


!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
!pip install --no-cache-dir -I pillow



# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

import torch

!mkdir -p drive
!google-drive-ocamlfuse drive
!pip install --no-cache-dir -I pillow


import os
print(os.listdir(os.getcwd()))
os.chdir('drive/AI/B')
print(os.listdir(os.getcwd()))




labels_to_atrs_dict = {}
train_pics_dict={}
train_pics_to_attr_dict = {}
test_pics_list = []

from sklearn import tree
import numpy as np
from PIL import Image
def register_extension(id, extension): Image.EXTENSION[extension.lower()] = id.upper()
Image.register_extension = register_extension
def register_extensions(id, extensions): 
  for extension in extensions: register_extension(id, extension)
Image.register_extensions = register_extensions

import pickle




def load_label(file_name):
    with open(file_name) as labels:
        content = labels.read().split('\n')
    label_list = [x.split('\t')[0] for x in content][:-1]
    return label_list

def load_label_attr(file_name):
    with open(file_name) as labels:
        content = labels.read().split('\n')
    label_attr = [list(map(float,x.split('\t')[1:])) for x in content][:-1]
    atrs = np.array(label_attr[0])
    label_attr = label_attr[1:]
    for i in label_attr:
        ii = np.array(i)
        atrs = np.vstack((atrs,ii))
    return atrs

def create_label_to_atr_dict():
    labels_list = load_label('attributes_per_class.txt')
    atrs_list = load_label_attr('attributes_per_class.txt')
    for label in labels_list:
        labels_to_atrs_dict[label] = atrs_list[labels_list.index(label)]

def label_to_atr(label):
    return labels_to_atrs_dict[label]
import math
def newAttr_to_label(atr):  
    maxProduct = 0;
    bestKey = 'ZJL1'
    for key,value in labels_to_atrs_dict.items():
        p = 1
        for i in range(len(value)):
            p = p * (math.fabs(atr[i] - value [i]))
        if p > maxProduct :
            maxProduct = p
            bestKey = key
    return bestKey

def atr_to_label(atr):
    for key,value in labels_to_atrs_dict.items():
        try:
            if set(value) == set(atr):
                return key
        except:
            print('key is:', key, 'value shape is:', value.shape, 'atr shape is:', atr.shape)
    tree = dsc_tree_train(load_label_attr('attributes_per_class.txt'),load_label('attributes_per_class.txt'))
    tst_atr = np.array([atr])
    return dsc_tree_prediction(tree,tst_atr)

def create_train_pics_dict():
    file_name = "train.txt"
    with open(file_name) as pics_labels:
        content = pics_labels.read().split('\n')
    pics_list = [x.split('\t')[0] for x in content][:-1]
    lables_list = [x.split('\t')[1:] for x in content][:-1]
    for pic in pics_list:
        train_pics_dict[pic] = lables_list[pics_list.index(pic)][0]
def create_test_pics_list():
    file_name = "image.txt"
    with open(file_name) as pics_labels:
        a = pics_labels.read().split('\n')[:-1]
        test_pics_list.extend(a)
def load_pic(pic_name,train_flag):
    if train_flag:
        pic = "train/"+pic_name
    else:
        pic = "test/"+pic_name
    return Image.open(pic)

def create_train_pic_atr_dict():
    for pic,label in train_pics_dict.items():
        train_pics_to_attr_dict[pic] = labels_to_atrs_dict[label]


def dsc_tree_train(X,y):
    attr_clf = tree.DecisionTreeClassifier()
    attr_clf.fit(X,y)
    return attr_clf

def dsc_tree_prediction(clf,X):
    return clf.predict(X)

def save_obj(obj, name ):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

def initialize():
    global labels_to_atrs_dict, train_pics_to_attr_dict, train_pics_dict
    try:
        labels_to_atrs_dict = np.load('labels_to_atrs_dict.npy').items()
    except:
        create_label_to_atr_dict()
        np.save('labels_to_atrs_dict.npy',labels_to_atrs_dict)
    try:
        train_pics_dict = np.load('train_pics_dict.npy').items()
    except:
        create_train_pics_dict()
        np.save('train_pics_dict.npy',train_pics_dict)
    try:
        train_pics_to_attr_dict = np.load('train_pics_to_attr_dict.npy').items()
    except:
        create_train_pic_atr_dict()
        np.save('train_pics_to_attr_dict.npy',train_pics_to_attr_dict)
# def write_submition(img_name,label,write_in_new_file_flag):

import random
from os import walk
from os import path
from PIL import Image
from torchvision import datasets, transforms
import torch

normalize = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)
preprocess = transforms.Compose([
    transforms.Resize(64),
    transforms.ToTensor(),
    normalize
])

def LoadData(dir, trainPercentage, validSize):
    train = []
    trainLabels = []
    validation = []
    validLabels = []
    i = 0
    for (dirpath, dirnames, filenames) in walk(dir):
        total = len(filenames)
        random.shuffle(filenames)
        for filename in filenames:
            filepath = path.join(dirpath, filename)
            imgCV = cv.imread(filepath)
            #impp = np.reshape(img,(img.shape[2],img.shape[0],img.shape[1]))
            img = Image.fromarray(imgCV)
            
            #print(impp.shape[0])
            if imgCV.shape[2] != 3:
                continue
            
            img_tensor = preprocess(img)
            if(i%100 == 0):
                print("loaded: ",i)
            if(i <= int(trainPercentage * total)):
                train.append(img_tensor)
                trainLabels.append(filename)
            elif len(validation) <  validSize:
                validation.append(img_tensor)
                validLabels.append(filename)
            else:
                break
            i = i + 1
    return train, trainLabels, validation, validLabels

  
trainImgs = []
trainNames = []
validImgs =[]
validNames =[]
(trainImgs,trainNames,validImgs,validNames) = LoadData('train',0.7,10000)

initialize()

####################### train
attrList = []
for trainName in trainNames:
   attr = train_pics_to_attr_dict[trainName]
   attrList.append(attr)

target = torch.FloatTensor(attrList)
print("kheili alaki")
def load_train_images(image_size=64, batch_size=64, root="./img/"):

    train_set = trainImgs

    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=False, num_workers=1)
    #train_loader.dataset.target = torch.FloatTensor([0,1])
    return train_loader

####################### validation
attrListValid = []
for validName in validNames:
   attr = train_pics_to_attr_dict[validName]
   attrListValid.append(attr)
targetValid = torch.FloatTensor(attrListValid)
print("kheili alaki")
def load_train_images_validation(image_size=64, batch_size=64, root="./img/"):
    train_set = validImgs
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=False, num_workers=1)
    #train_loader.dataset.target = torch.FloatTensor([0,1])
    return train_loader
  
  
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import numpy as np

import os


#print(attr)
torch.set_num_threads(16)
train_loader = load_train_images()
train_loader_valid = load_train_images_validation()
####### CNN

targetLearnSize = 30

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=50):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(2, stride=1)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

#models = [ResNet(Bottleneck, [3, 8, 36, 3]) for _ in range(1)]
#
model = ResNet(Bottleneck, [3, 8, 36, 3])
model.cuda()

#optimizer = torch.optim.Adam(model.parameters(),lr=0.01, betas=(0.9, 0.999))
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

def train(epoch):
    print("train started")
    global train_loader
    model.train()
    # try:
    for batch_idx,data in enumerate(train_loader):
        #print("in train loop   ",batch_idx)
        #data, target = Variable(data), Variable(target)
        data = Variable(data).cuda()
        optimizer.zero_grad()
        output = model(data)
        #print(output)
        criterion = nn.MSELoss()
        try:
            loss = criterion(output, Variable(target[batch_idx*64:(batch_idx+1)*64]).cuda())
        except:
            loss = criterion(output, Variable(target[batch_idx * 64: ]).cuda())
        loss.backward()
        #print(model.conv1.bias.grad)

        optimizer.step()

        if(batch_idx% 10 == 0) :
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))
    # except:
    #     for img in train_loader.dataset:
    #         if img.shape != torch.Size([3,64,64]):
    #             ews = type(train_loader.dataset)
    #             idvv = train_loader.dataset.index(img)
    #             print(trainNames[train_loader.dataset.index(img)])
correctCounter = 0


def test():
    global correctCounter
    correctCounter = 0
    for batch_idx,data in enumerate(train_loader_valid):
        data = Variable(data).cuda()
#        s = data.detach().numpy().shape[0]
 #       testOP = np.empty([s, 30])
#         for i in range(len(models)):
#             output = models[i](data)
#             output = output.detach().numpy()
#             testOP[:, i] = output.flatten()
        output = model(data).cpu().detach().numpy()
        
        print(batch_idx)
        counter = 0
        for op in output:
            # m = atr_to_label(op)
            # print(m ,'::::' ,validNames[counter],"::::",train_pics_dict[validNames[counter]])
#             print(newAttr_to_label(op))
#             print(train_pics_dict[validNames[counter +  batch_idx*64]])
#             print('test',op)
#             print('real',train_pics_to_attr_dict[validNames[counter +  batch_idx*64]])
    
            if newAttr_to_label(op) == train_pics_dict[validNames[counter +  batch_idx*64]]:
                correctCounter = correctCounter + 1
                print(newAttr_to_label(op))
                print(train_pics_dict[validNames[counter +  batch_idx*64]])
            
            counter = counter + 1
            
try:            
    model = torch.load("Save/model.pt")
except:
    print('new model')
print(train_loader)
for epoch in range(1, 10000):
    train(epoch)
    torch.save(model, "Save/model"+ ".pt")
    if epoch%100 == 0:
        test()
        print("this is correct num:",correctCounter,"Pers:",correctCounter/(len(validNames)))
test()
print("this is correct num:",correctCounter,"Pers:",correctCounter/(len(validNames)))

print("alaki")






Redirecting output to ‘wget-log.59’.
(Reading database ... 19845 files and directories currently installed.)
Preparing to unpack google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.0-0ubuntu1) over (0.7.0-0ubuntu1) ...
Setting up google-drive-ocamlfuse (0.7.0-0ubuntu1) ...
Reading package lists... Done
Building dependency tree       
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
··········
    100% |████████████████████████████████| 2.0MB 2.6MB/s 
    100% |████████████████████████████████| 2.0MB 57.4MB/s 
['google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb.29', 'google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb.6', 'wget-log.52', 'wget-log.58', 'wget-log.28', 'google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb.49', 'google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb.13', 'wget-log.41', 'wget-log.42', 'google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb.45', 'wget-log.10', 'google-drive-ocamlfuse_0.7.0-0ubuntu1

# New Section